In [5]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import stem
import math
import operator

In [6]:
# carregando os dados
url = "https://raw.githubusercontent.com/LiviaCavalcanti/RecInfo/master/lab02/results.csv"
original_data = pd.read_csv(url)
N_TOTAL_DOCUMENTS = original_data.shape[0]
original_data.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


In [7]:
# gerando tokens
array_df_token = []
original_data.title.apply(lambda news: array_df_token.append( re.split(r'\W+', news)))
array_df_token[0]

['',
 'A',
 'sociedade',
 'foi',
 'Rubens',
 'Paiva',
 'não',
 'os',
 'facínoras',
 'que',
 'o',
 'mataram',
 '']

In [8]:

# para o first time run
nltk.download('stopwords')

# removendo stopwords
default_stopwords = set(stopwords.words('portuguese'))
stopwords_complement = ["", "neste","és", "é", "somos", "sois", "são", "era", "eras", "éreis", "foste", "fostes", "foras",	"fôreis",	"serás", "sereis", "serão",	"serias", "seríeis", "sejas", "sejais", "fosses", "fôsseis", "fores", "fordes", "sê", "sejas", "sejais", "ser", "seres", "sermos",	"serdes",	"serem", "sobre", "é"]

stopwords_complement.extend(default_stopwords)



[nltk_data] Downloading package stopwords to /home/livia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
stopwords_complement.extend(default_stopwords)


In [10]:
no_stopwords_news = []
for word_array in array_df_token:
    no_stopwords_news.append(list(filter(lambda x : x.lower() not in stopwords_complement, word_array)))

In [11]:
filtered_news = []

for e in no_stopwords_news:
    aux_word = []
    for word in e:
    aux_word.append(word.lower())
    filtered_news.append(aux_word[:10])

filtered_news

In [24]:
filtered_news


[['sociedade', 'rubens', 'paiva', 'facínoras', 'mataram'],
 ['justiça',
  'suspende',
  'decisão',
  'proibia',
  'forças',
  'armadas',
  'celebrarem',
  'golpe',
  '1964'],
 ['governo', 'bolsonaro', 'prega', 'negacionismo', 'histórico', 'ditadura'],
 ['pais', 'gabo', 'perceberam', 'filho', 'mentiroso'],
 ['rádios',
  'canadenses',
  'banem',
  'músicas',
  'michael',
  'jackson',
  'programação'],
 ['rosângela',
  'crianças',
  'sequestradas',
  'famílias',
  'militares',
  'ditadura'],
 ['lógica', 'usar', 'torturadores', 'ditadura', 'crime', 'usada', 'milícias'],
 ['bolsonaro',
  'troca',
  'embaixada',
  'escritório',
  'jerusalém',
  'evita',
  'retaliação',
  'palestina'],
 ['lado', 'sombrio', 'dickens'],
 ['amor', 'mulheres', 'sacudiu', 'paraguai'],
 ['feminista', 'susan', 'sontag', 'contra', 'norman', 'mailer'],
 ['imagens', 'questionadoras', 'harun', 'farocki'],
 ['manuscrito',
  'cem',
  'anos',
  'solidão',
  'garcía',
  'márquez',
  'acreditava',
  'ter',
  'perdido'],
 ['r

In [47]:
from wikipedia2vec import Wikipedia2Vec
wiki2vec = Wikipedia2Vec.load('ptwiki_20180420_100d.pkl')
wiki2vec.most_similar(wiki2vec.get_word('lívia'), 5)

[(<Word lívia>, 1.0),
 (<Word júlia>, 0.8488017),
 (<Word valéria>, 0.8453937),
 (<Word patrícia>, 0.83301896),
 (<Word priscila>, 0.81010646)]

In [26]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('cbow_s50.txt')

In [27]:
model["saber"]

array([ 0.247879, -0.202555,  0.261191, -0.414436, -0.393175, -0.133628,
        0.331779,  0.490606,  0.223101,  0.050068,  0.073744, -0.028905,
       -0.270805,  0.107943, -0.120106, -0.03977 , -0.298455, -0.203383,
       -0.231529, -0.133759, -0.487622,  0.215582,  0.033018,  0.343666,
        0.256141,  0.555295,  0.165039,  0.005324, -0.029864,  0.052081,
        0.179733, -0.061053, -0.137913, -0.307359,  0.06872 , -0.274085,
       -0.150493,  0.068434, -0.032436, -0.107531,  0.215385,  0.030889,
        0.336944, -0.148462,  0.041685,  0.239185,  0.132592,  0.210251,
        0.090691, -0.316316], dtype=float32)

In [46]:
model.similarity("similar", "semelhante")

0.9536234943063773